# Part 5 - Multiple Linear Regression

In the previous notebook we trained a Simple Linear Regressor with a single feature (`sqft`).  
In this notebook we aim to extend the capability of our model by using multiple features for our independent variable `X`.  
Our equation is the same however our `X` is now a matrix and the equation no longer represents a line but rather a hyperplane in N-dimensional space where **N** is the number of features in `X`.
  
$$
y = mX + b
$$ 
  


In [ ]:
import time
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
%matplotlib inline

## Let's load the data and remind ourselves of the contents

In [ ]:
df = pd.read_csv('./data/sf/data_clean_engineered.csv')
df.head()

## Prepare our training and validation data

In [ ]:
df.columns

Recall we need to reshape the input data

In [ ]:
features = [feature for feature in df.columns if feature != 'price']
X = df[features]
y = df['price']
X_np = X.values
y_np = y.values.reshape((len(df), 1))

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_np, y_np, test_size=0.30, random_state=2018) # split 70% train, 30% validation

In [ ]:
regressor = LinearRegression(normalize=True)

In [ ]:
model = regressor.fit(X_train, y_train)

In [ ]:
def evaluate_model(model, X, y):
    y_pred = model.predict(X) # predict y values from input X
    mse = mean_squared_error(y_true=y, y_pred=y_pred)
    print("Mean Squared Error: {}".format(mse))
    print("Accuracy: {}%".format(model.score(X, y)*100.0))
evaluate_model(model, X_val, y_val)

In [ ]:
# try brand new data
actual_price = '$628,000'
sqft = 425
bed = 0
bath = 1
property_type = 'condo'
postal_code = '94103'
new_data = {'sqft': sqft,
            'bed': bed,
            'bath': bath,
            'property_type_{}'.format(property_type): 1,
            'postal_code_{}'.format(postal_code): 1
           }
new_df = pd.get_dummies(pd.DataFrame(data=[new_data], columns=X.columns).fillna(0))
new_df
print(X.shape)
predicted_price = model.predict(new_df).squeeze()
print("predicted price: ${}M".format(predicted_price/1e6))
print("actual price: {}".format(actual_price))


Compare with our simple linear model from before

In [ ]:
with open('./models/sf/simple_linear.pkl', 'rb') as f:
    simple_linear = pickle.load(f)
# print(prediction)
sqft_input = np.array([sqft]).reshape(1,1)
predicted_price_sl = simple_linear.predict(sqft_input).squeeze()
print("simple linear predicted price: ${}M".format(predicted_price_sl/1e6))

## Retrain on entire dataset and save model to disk

In [ ]:
model = regressor.fit(X, y)
with open('./models/sf/multiple_linear.pkl', 'wb') as f:
    pickle.dump(model, f)